## Craremos el dataframe para la funcion 6, utilizando el formato item-item


Comenzaremos por la importacion de las librerias pertinentes y luego montamos lo archivos para empezar a trabajar

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import nltk
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
ruta_items = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_items.csv'
ruta_juegos = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_games.csv'
ruta_reviews = '/content/drive/MyDrive/TP N1 Henry/EDA/steam_reviews.csv'


pd_items = pd.read_csv(ruta_items)
pd_reviews = pd.read_csv(ruta_reviews)
pd_games = pd.read_csv(ruta_juegos)

<ipython-input-3-6ae08b9fba24>:7: DtypeWarning: Columns (1,2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_reviews = pd.read_csv(ruta_reviews)


Tomamos las columnas que nos interesan de cada uno de los 3 dataframes

In [4]:
red_items = pd_items.groupby('user_id')['item_id'].agg(list).reset_index()
red_items.columns = ['user_id', 'item_id_list']
red_items

,user_id,item_id_list
0,--000--,"[4000, 1250, 35420, 550, 223530, 43110, 105600..."
1,--ace--,"[4000, 23490, 40800, 620, 113200, 205790, 730,..."
2,--ionex--,"[10180, 10190, 550, 223530, 620, 105600, 42680..."
3,-2SV-vuLB-Kg,"[29800, 104200, 730, 200510, 254040, 238910, 2..."
4,-404PageNotFound-,"[240, 4000, 10180, 10190, 550, 223530, 8190, 3..."
...,...,...
70907,zzonci,"[2400, 2420, 2430, 205790, 216150]"
70908,zzoptimuszz,"[205790, 99900, 104700, 215120, 230410, 243870..."
70909,zzydrax,"[240, 4000, 10090, 24240, 212500, 237310, 2656..."
70910,zzyfo,"[4000, 8000, 9480, 23310, 24740, 11390, 24800,..."


In [5]:
games_red = pd.DataFrame({
    'id': pd_games['id'].astype(int),
    'app_name': pd_games['app_name'],
    'genre': pd_games['genres'],
    'price': pd_games['price']
})
games_red

,id,app_name,genre,price
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']",0.00
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",0.00
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']",0.99
4,773570,Log Challenge,Desconocido,2.99
...,...,...,...,...
32130,773640,Colony On Mars,"['Casual', 'Indie', 'Simulation', 'Strategy']",1.99
32131,733530,LOGistICAL: South Africa,"['Casual', 'Indie', 'Strategy']",4.99
32132,610660,Russian Roads,"['Indie', 'Racing', 'Simulation']",1.99
32133,658870,EXIT 2 - Directions,"['Casual', 'Indie']",4.99


In [7]:
reviews_red = pd.DataFrame({
    'user_id': pd_reviews['user_id'],
    'item_id': pd_reviews['item_id'].dropna(),
    'review': pd_reviews['review'].dropna()
})

In [8]:
reviews_red_sin_nulos = reviews_red.dropna(how='any')

reviews_red = reviews_red_sin_nulos

In [9]:
reviews_red.loc[:, 'item_id'] = reviews_red['item_id'].astype(int)


<ipython-input-9-f06d13a11fc9>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  reviews_red.loc[:, 'item_id'] = reviews_red['item_id'].astype(int)


luego de quitar nulos, cambiar formatos y preparar los demas dataframes, nos centraremos en el de items, haremos un preprocesado a la columna reviews para obtener un sentimiento de analisis preciso

In [10]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
    filtered_tokens = [token for token in tokens if token not in stop_words and token.isalpha()]  # Eliminación de stopwords y signos de puntuación
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lematización
    return lemmatized_tokens

In [11]:
reviews_red['processed_reviews'] = reviews_red['review'].apply(preprocess_text)

In [12]:
def analyze_sentiment_textblob(text):
    if isinstance(text, list) and text:
        try:
            text_str = " ".join(text)
            blob = TextBlob(text_str)
            sentiment_score = blob.sentiment.polarity
            return sentiment_score
        except Exception as e:
            print("Error:", e)
            print("Text causing the error:", text)
            return 0.0
    else:
        return 0.0


In [13]:
reviews_red['sentiment_score_textblob'] = reviews_red['processed_reviews'].apply(analyze_sentiment_textblob)


In [14]:
reviews_red

,user_id,item_id,review,processed_reviews,sentiment_score_textblob
0,76561197970982479,1250,Simple yet with great replayability. In my opi...,"[simple, yet, great, replayability, opinion, z...",0.211111
1,76561197970982479,22200,It's unique and worth a playthrough.,"[unique, worth, playthrough]",0.337500
2,76561197970982479,43110,Great atmosphere. The gunplay can be a bit chu...,"[great, atmosphere, gunplay, bit, chunky, time...",0.075000
3,js41637,251610,I know what you think when you see this title ...,"[know, think, see, title, barbie, dreamhouse, ...",0.057407
4,js41637,227300,For a simple (it's actually not all that simpl...,"[simple, actually, simple, truck, driving, sim...",-0.036111
...,...,...,...,...,...
263221,76561198312638244,70,a must have classic from steam definitely wort...,"[must, classic, steam, definitely, worth, buying]",0.233333
263222,76561198312638244,362890,this game is a perfect remake of the original ...,"[game, perfect, remake, original, half, life, ...",0.167188
263223,LydiaMorley,273110,had so much fun plaing this and collecting res...,"[much, fun, plaing, collecting, resource, xd, ...",0.087500
263224,LydiaMorley,730,:D,[],0.000000


Quitamos la scolumnas que quedaron obsoletas y nos quedaremos con

- iD (de los juegos)

- positivo, neutral, negativo (cada columna contiene la sumatoria de reviews positivas, neutrales y negativas para cada juego)

In [15]:
columnas = ['review','processed_reviews']
reviews_red = reviews_red.drop(columns = columnas)
reviews_red.rename(columns={'item_id': 'id'}, inplace=True)


In [16]:
def convertir_sentimiento(valor):
    if valor < -0.3:
        return 0
    elif -0.3 <= valor <= 0.3:
        return 1
    else:
        return 2

reviews_red['sentiment_score_textblob'] = reviews_red['sentiment_score_textblob'].apply(lambda x: convertir_sentimiento(x))

In [17]:
reviews_red2= reviews_red.groupby('id')['sentiment_score_textblob'].value_counts().unstack(fill_value=0)
reviews_red2.columns = ['negativo', 'neutral', 'positivo']
reviews_red2.reset_index(inplace=True)
reviews_red2

,id,negativo,neutral,positivo
0,10,3,44,9
1,20,1,13,3
2,30,0,3,1
3,40,0,0,1
4,50,0,4,0
...,...,...,...,...
3677,521340,0,1,1
3678,521430,0,1,0
3679,521570,0,2,0
3680,521990,0,0,1


In [18]:
items_red = pd.DataFrame({
    'user_id': pd_items['user_id'],
    'id': pd_items['item_id'],
    'playtime_forever': pd_items['playtime_forever'],
})

In [19]:
games_red.head()

,id,app_name,genre,price
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']",0.00
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",0.00
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']",0.99
4,773570,Log Challenge,Desconocido,2.99


In [20]:
items_red.head()

,user_id,id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0


Unimos todo, para tener el primer esquema de nuestro dataframe a utilizar

In [44]:
merged_df = pd.merge(reviews_red2, games_red, on='id', how='inner')
merged_df = pd.merge(merged_df, items_red, on=['id'], how='inner')
merged_df.rename(columns={'sentiment_score_textblob': 'SCB'}, inplace=True)
merged_df = merged_df[['app_name', 'positivo','neutral','negativo', 'playtime_forever', 'genre', 'price']]
merged_df = merged_df.drop_duplicates()

In [45]:
agregaciones = {
    'positivo': 'first',
    'neutral': 'first',
    'negativo': 'first',
    'playtime_forever': 'sum',
    'genre': 'first',
    'price': 'first'
}
merged_df = merged_df.groupby('app_name').agg(agregaciones).reset_index()
merged_df

,app_name,positivo,neutral,negativo,playtime_forever,genre,price
0,! That Bastard Is Trying To Steal Our Gold !,0,1,0,742,"['Action', 'Adventure', 'Casual', 'Indie']",3.99
1,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,1,0,15692,"['Action', 'Indie']",3.99
2,0RBITALIS,0,1,0,12438,"['Indie', 'Simulation']",9.99
3,"10,000,000",0,3,0,128680,"['Action', 'Casual', 'Indie', 'RPG']",4.99
4,100% Orange Juice,0,10,4,446717,"['Indie', 'Strategy']",6.99
...,...,...,...,...,...,...,...
2923,the static speaks my name,0,13,3,2467,"['Adventure', 'Free to Play', 'Indie', 'RPG']",0.00
2924,theBlu,1,0,0,3335,Desconocido,9.99
2925,theHunter Classic,8,56,14,1050386,"['Action', 'Adventure', 'Free to Play', 'Simul...",0.00
2926,theHunter: Primal,1,13,2,163901,"['Action', 'Adventure', 'Simulation']",9.99


analizaremos la columna genre

desanidaremos las listas de la columna y convertiremos algunos valores en semejantes para reducir la cantidad de variables para cuando toque hacer One Hot Encoding


In [46]:
dat_func6 = merged_df

In [47]:
dat_func6['genre'] = dat_func6['genre'].str.strip("[]").astype(str)

In [48]:
dat_func6['genre'] = dat_func6['genre'].str.replace("'", "")

In [49]:
cadena_genre = dat_func6['genre'].tolist()
palabras_genre = [palabra.strip("'") for cadena in cadena_genre for palabra in cadena.split(", ")]
valores_unicos_genre = set(palabras_genre)

valores_unicos_genre

{'Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Desconocido',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

hay varios valores agrupables dentro del valor Utilities, Free to Play ya esta explicito en el precio y Early Access es un valor inutil para nuestro objetivo

In [50]:
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Animation\s?&amp;\s?Modeling', 'Utilities', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Design\s?&amp;\s?Illustration', 'Utilities', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Video Production', 'Utilities', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Web Publishing', 'Utilities', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Software Training', 'Utilities', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Audio Production', 'Utilities', regex=True)

dat_func6['genre'] = dat_func6['genre'].str.replace(r'Free to Play', 'delete', regex=True)
dat_func6['genre'] = dat_func6['genre'].str.replace(r'Early Access', 'delete', regex=True)

In [51]:
dat_func6.head()

,app_name,positivo,neutral,negativo,playtime_forever,genre,price
0,! That Bastard Is Trying To Steal Our Gold !,0,1,0,742,"Action, Adventure, Casual, Indie",3.99
1,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,1,0,15692,"Action, Indie",3.99
2,0RBITALIS,0,1,0,12438,"Indie, Simulation",9.99
3,"10,000,000",0,3,0,128680,"Action, Casual, Indie, RPG",4.99
4,100% Orange Juice,0,10,4,446717,"Indie, Strategy",6.99


Quitamos los espacios para que no haya problemas a la hora de realizar el OHC

In [52]:
dat_func6['genre'] = dat_func6['genre'].str.replace(r'\s*,\s*', ',', regex=True)
dat_func6

,app_name,positivo,neutral,negativo,playtime_forever,genre,price
0,! That Bastard Is Trying To Steal Our Gold !,0,1,0,742,"Action,Adventure,Casual,Indie",3.99
1,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,1,0,15692,"Action,Indie",3.99
2,0RBITALIS,0,1,0,12438,"Indie,Simulation",9.99
3,"10,000,000",0,3,0,128680,"Action,Casual,Indie,RPG",4.99
4,100% Orange Juice,0,10,4,446717,"Indie,Strategy",6.99
...,...,...,...,...,...,...,...
2923,the static speaks my name,0,13,3,2467,"Adventure,delete,Indie,RPG",0.00
2924,theBlu,1,0,0,3335,Desconocido,9.99
2925,theHunter Classic,8,56,14,1050386,"Action,Adventure,delete,Simulation,Sports",0.00
2926,theHunter: Primal,1,13,2,163901,"Action,Adventure,Simulation",9.99


In [53]:
genre_sep = dat_func6['genre'].str.get_dummies(sep=',')
dat_func6_sep = pd.concat([dat_func6, genre_sep], axis=1)
dat_func6_sep.drop('genre', axis=1, inplace=True)

In [54]:
dat_func6_sep

,app_name,positivo,neutral,negativo,playtime_forever,price,Action,Adventure,Casual,Desconocido,Education,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,Utilities,delete
0,! That Bastard Is Trying To Steal Our Gold !,0,1,0,742,3.99,1,1,1,0,0,1,0,0,0,0,0,0,0,0
1,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,1,0,15692,3.99,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0RBITALIS,0,1,0,12438,9.99,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,"10,000,000",0,3,0,128680,4.99,1,0,1,0,0,1,0,1,0,0,0,0,0,0
4,100% Orange Juice,0,10,4,446717,6.99,0,0,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,the static speaks my name,0,13,3,2467,0.00,0,1,0,0,0,1,0,1,0,0,0,0,0,1
2924,theBlu,1,0,0,3335,9.99,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2925,theHunter Classic,8,56,14,1050386,0.00,1,1,0,0,0,0,0,0,0,1,1,0,0,1
2926,theHunter: Primal,1,13,2,163901,9.99,1,1,0,0,0,0,0,0,0,1,0,0,0,0


In [55]:
dat_func6_sep.drop(columns = 'delete', inplace = True)


In [56]:
dat_func6_sep.to_csv('recomendacion_juego.csv', index = False)